# Assignment: Extracting Static WebPage

Extract information about “วันพระ” for 3 years from:
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx


Note that you can use dateparse package to parse Thai date.  First, we will have to install the package, this is for Google Colab users.  Otherwise, installing via command line is recommended (pip or conda).

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install dateparser

In [2]:
import dateparser

To convert from thai text date string, we will use the parse method.  Note that the parse method assumes the BC year, not BE.  Thus, we will have to subtract 543 from the year.  In addition, weekday() returns day of week with 0=Monday, ..., 6=Sunday.

In [3]:
dt = dateparser.parse('วันศุกร์ที่ 17 มกราคม 2563')

# this will print out weekday == 0 (Monday)
print(dt)
print(dt.weekday())

# this will print out weekday == 4 (Friday)
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2563-01-17 00:00:00
0
2020-01-17 00:00:00
4


In [4]:
dt = dateparser.parse('วันเสาร์ที่ 21 กันยายน 2564')
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2021-09-21 00:00:00
1


Count the distribution of number of week days that are “วันพระ” for all three years and answer the following questions:

## How many วันพระ in total (of 3 years)?

In [5]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
day_map = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

In [7]:
data2565 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx')
data2566 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx')
data2567 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx')

In [8]:
soup2565 = BeautifulSoup(data2565.text, "lxml")
soup2566 = BeautifulSoup(data2566.text, "lxml")
soup2567 = BeautifulSoup(data2567.text, "lxml")

In [9]:
print(soup2565.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   วันพระ 2565 ปฏิทินวันพระ พ.ศ.2565/2022 - myhora.com
  </title>
  <meta content="วันพระ 2565 ปฏิทินวันพระ 2565/ 2022 ปฏิทินวันพระตลอดปี 2565 วันธรรมสวนะหรือวันอุโบสถ ได้แก่ วันขึ้น ๘ ค่ำ, วันวันเพ็ญขึ้น ๑๕ ค่ำ , วันแรม ๘ ค่ำ และวันแรม ๑๕ ค่ำ หากเดือนใดเป็นเดือนขาด ถือเอาวันแรม ๑๔ ค่ำ วันมาฆบูชา วิสาขบูชา วันอาสาฬหบูชา วันเข้าพรรษา วันออกพรรษา" name="Description"/>
  <meta content="ปฏิทินวันพระ 2565,วันพระ 2022,วันพระ พ.ศ.2565,Calendar 2022,ปฏิทินปี 2565,ปฏิทินจันทรคติไทย พ.ศ.2565,ปฏิทินวันพระ 2565,ปฏิทินวันหยุดราชการ 2565 ,วันหยุดชดเชย,วันพระ 8 ค่ำ,วันพระ 15 ค่ำ,วันขึ้นปีใหม่,วันมาฆบูชา 2565,วิสาขบูชา 2565,วันอาสาฬหบูชา 2565,วันเข้าพรรษา 2565,วันออกพรรษา 2565,วันลอยกระทง 2565, Calendar 2022" name="Keywords"/>
  <meta content="10 Days" name="Revisit-After"/>
  <meta content="index,follow" name="robots"/>
  <meta content="noodp" name="robots"/>
  <meta content="MYHORA.com, Thailand" name="author"/>
  <meta content="787285358050657" property="fb:

In [10]:
def conver_to_df(soup):
  processed_data = [{
      'original': x.div.text,
      'date': (date := dateparser.parse(x.div.text).replace(year=dateparser.parse(x.div.text).year - 543)),
      'weekday': day_map[date.weekday()]
      } for x in soup.find_all(attrs={'class': 'bud-day'})]
  return pd.DataFrame(processed_data, columns=['original', 'date', 'weekday'])

df_2565 = conver_to_df(soup2565)
df_2566 = conver_to_df(soup2566)
df_2567 = conver_to_df(soup2567)

df = pd.concat([df_2565, df_2566, df_2567], ignore_index=True)

df.head()

,original,date,weekday
0,วันอาทิตย์ที่ 2 มกราคม 2565,2022-01-02,Sunday
1,วันจันทร์ที่ 10 มกราคม 2565,2022-01-10,Monday
2,วันจันทร์ที่ 17 มกราคม 2565,2022-01-17,Monday
3,วันอังคารที่ 25 มกราคม 2565,2022-01-25,Tuesday
4,วันอังคารที่ 1 กุมภาพันธ์ 2565,2022-02-01,Tuesday


In [11]:
print(f"All วันพระ in total (of 3 years): {len(df)}")

All วันพระ in total (of 3 years): 152


## How many days in total (of 3 years) that วันพระ is Monday?

In [12]:
def find_day(df, day):
  return len(df[df['weekday'] == day])

In [13]:
print(f"All days in total (of 3 years) that วันพระ is Monday: {find_day(df, 'Monday')}")

All days in total (of 3 years) that วันพระ is Monday: 21


## Which day of the week that has the minimum number of วันพระ?

In [14]:
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
count_day = {}
for day in days:
  count_day[day] = find_day(df,day)
print(count_day)

{'Sunday': 24, 'Monday': 21, 'Tuesday': 20, 'Wednesday': 22, 'Thursday': 23, 'Friday': 21, 'Saturday': 21}


In [15]:
print(f"{min(count_day, key=count_day.get)} is day of the week that has the minimum number of วันพระ")

Tuesday is day of the week that has the minimum number of วันพระ


## Which day of the week that has the maximum number of วันพระ?

In [16]:
print(f"{max(count_day, key=count_day.get)} is day of the week that has the maximum number of วันพระ")

Sunday is day of the week that has the maximum number of วันพระ
